Source https://www.kaggle.com/thomasmichel/use-models-from-dataset

In [1]:
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
tqdm.pandas()

from shopee.image import get_embeddings
from shopee.image.data import ImageInferenceDataset

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using the following device: {device}")

Using the following device: cuda


In [2]:
SHOPEE_DATA_PATH = '/opt/jsk/share/shopee-data/'

In [3]:
COMPUTE_CV = False

test = pd.read_csv(f'{SHOPEE_DATA_PATH}/test.csv')
if len(test)>3: COMPUTE_CV = False
else: print('this submission notebook will compute CV score, but commit notebook will not')

this submission notebook will compute CV score, but commit notebook will not


In [4]:
PATH = '/opt/jsk/share/shopee-models/'

In [5]:
model_name = "efficientnet_b0"
weights_paths = {
    "pretrained": f"{PATH}/hub/checkpoints/efficientnet_b0_ra-3dd342df.pth",
    "arcface_triplets": f"{PATH}/image/efficientnet/efficientnet_b0_finetuned_arcface_triplets.pth"
}

weights = "arcface_triplets"

# load model architecture
image_model = torch.hub.load(
    f"{PATH}/hub/rwightman_gen-efficientnet-pytorch_master/",
    model_name,
    pretrained=False,
    source="local"
)

# load pre-trained weights
image_model.load_state_dict(torch.load(
    weights_paths["pretrained"],
    map_location="cpu"
))

# remove last layer (classification layer)
image_model = nn.Sequential(*list(image_model.children())[:-1])

if weights == "arcface_triplets":
    image_model.load_state_dict(torch.load(
            weights_paths[weights],
            map_location="cpu"
        ))

image_model.eval()
image_model.to("cpu")
pass

In [6]:
if COMPUTE_CV:
    data = pd.read_csv(f'{SHOPEE_DATA_PATH}/train.csv')
    images_path = f'{SHOPEE_DATA_PATH}/train_images/'
    print('Using train as test to compute CV (since commit notebook). Shape is', data.shape )
else:
    data = pd.read_csv(f'{SHOPEE_DATA_PATH}/test.csv')
    images_path = f'{SHOPEE_DATA_PATH}/test_images/'

In [7]:
batch_size = 64
num_workers = 2

image_embeddings = get_embeddings(
    data,
    images_path,
    image_model,
    batch_size,
    num_workers,
    device,
)

Computing image embeddings...



image embeddings shape torch.Size([3, 1280])
